In [2]:
from collections import deque
from aocutils.common import ints
from z3 import *


In [3]:
def neighbor(lights, button):
    return tuple(l if idx not in button else l ^ 1 for idx, l in enumerate(lights))

def bfs(lights, buttons, goal):
    # print(lights, buttons, goal)
    tocheck = deque([lights])
    seen = set()
    dis = 0
    while tocheck:
        for _ in range(len(tocheck)):
            cur = tocheck.popleft()
            for b in buttons:
                n = neighbor(cur, b)
                if n not in seen:
                    seen.add(n)
                    tocheck.append(n)
                    if n == goal:
                        return dis + 1
        dis += 1
    print(len(seen))

f = open("in.txt").read().strip().split("\n")
lines = [line for line in f]
lines[0]
ans = 0
for machine in lines:
    wiring, *buttons, joltage = machine.split()
    goal = tuple({'.':0, '#':1}[ch] for ch in wiring[1:-1])
    lights = tuple(0 for g in goal)
    buttons = [ints(b) for b in buttons]
    ans += bfs(lights, buttons, goal)
ans

449

In [4]:
ans = 0
for machine in lines:
    wiring, *buttons, joltage = machine.split()
    goal = tuple(ints(joltage))
    initjoltage = tuple(0 for _ in goal)
    buttons = [ints(b) for b in buttons]
    
    s = Optimize()
    z3_buttons = {}
    for idx, b in enumerate(buttons):
        exec(f'button{str(idx)} = Int("button{str(idx)}")')
        z3_buttons[idx] = eval(f'button{str(idx)}')
        s.add(z3_buttons[idx] >= 0)
    for idx, j in enumerate(goal):
        relevant_buttons = [i for i, b in enumerate(buttons) if idx in b]
        s.add(Sum([z3_buttons[i] for i in relevant_buttons]) == j)
    total = Int('tot')
    s.add(total ==  Sum([z3_buttons[i] for i in z3_buttons]) )
    s.minimize(total)
    if s.check() == sat:
        m = s.model()
        ans += m.eval(total).as_long()
ans

17848